In [1]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
import collections
from tika import parser
import csv
import sys
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.converter import XMLConverter, HTMLConverter, TextConverter
from pdfminer.layout import LAParams
import io
import numpy as np
import pandas as pd

# combined_words_for_tfidf = []
#This Function converts the Merged PDF to text
def pdfparser(data):

    fp = open(data, 'rb')
    rsrcmgr = PDFResourceManager()
    retstr = io.StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
# Create a PDF interpreter object.
    interpreter = PDFPageInterpreter(rsrcmgr, device)
# Process each page contained in the document.
    for page in PDFPage.get_pages(fp):
        interpreter.process_page(page)
        data =  retstr.getvalue()
    return data


In [ ]:
#By calling this function, it does all the jobs(PDF to text, Filtering Stop_words and creating a CSV file)
paths = [r'C:\Users\manee\Desktop\ADS\Assignment_1\Pdf_1.pdf',r'C:\Users\manee\Desktop\ADS\Assignment_1\Pdf_2.pdf',
         r'C:\Users\manee\Desktop\ADS\Assignment_1\Pdf_3.pdf',r'C:\Users\manee\Desktop\ADS\Assignment_1\Pdf_4.pdf']
combined_lists = []
data_lists = []
for path in paths:
    filtered_sentence = []
    content = pdfparser(path)
    data_lists.append(content)  


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(analyzer='word', min_df = 0, stop_words = 'english')

response = vectorizer.fit_transform(data_lists)

weights = np.asarray(response.mean(axis=0)).ravel().tolist()
weights_df = pd.DataFrame({'term': vectorizer.get_feature_names(), 'weight': weights})
weights_sorted = weights_df.sort_values(by='weight', ascending=False)
#if we need only the top 100 keywords within the file, use below
#weights_sorted = weights_df.sort_values(by='weight', ascending=False).head(100)

weights_sorted.to_csv(path_or_buf=r'C:\Users\manee\Desktop\ADS\Assignment_1\tfidf_WordList.csv')